In [1]:
import csv

In [2]:
folder = "benchmarks/flight/results"

approaches = ["dreach", "falstar", "enhsp", "enhsp_e_001", "enhsp_e_01", "enhsp_e_04"]
horizons = [2, 4, 6, 8, 10 , 20, 30, 40, 50, 60, 70, 80, 90, 100]

## Preprocess Falstar Summary CSV

In [3]:
line_count = 0
instances = range(10)
f = open("{}/falstar".format(folder), "w")

with open("{}/falstar.csv".format(folder), "r") as csvfile:
    csv_reader = csv.reader(csvfile, delimiter=',')
    next(csv_reader)
    
    for line in csv_reader:
              
        
        pnum = int(line_count%10)
        if pnum == 0:
            working_instances = []
            times = []
        if pnum in instances and int(line[3]) > 0: 
            #(f_fr[i]*f_avg_time[i]+(1-f_fr[i])*100)/f_fr[i]
            fr = int(line[3])/10
            avg_time = float(line[11])
            expected_time = (fr*avg_time + (1-fr)*100)/fr
            if expected_time < 300:
                working_instances += [pnum]
                times += [expected_time]
        
        if pnum == 9:
            f.write("{}\n".format(" ".join(map(str,times))))
            print(working_instances)
            
        line_count += 1
        
f.close()

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
[0, 4, 5, 9]
[4, 5, 9]
[4, 9]
[9]
[]


## Read results 

In [4]:
results = { app: {h:[] for h in horizons} for app in approaches}

for app in approaches:
    with open("{}/{}".format(folder, app), "r") as f:
        for h in horizons:
            app_results = f.readline()
            app_results = list(map(float,app_results.split()))
            results[app][h] = app_results
        


In [5]:
results

{'dreach': {2: [0.7765016555786133,
   0.8512752056121826,
   0.2326831817626953,
   0.2758486270904541,
   0.12030386924743652,
   0.1205453872680664,
   0.11610674858093262,
   0.13881683349609375,
   0.3520331382751465,
   0.3141210079193115],
  4: [13.8982994556427],
  6: [],
  8: [],
  10: [],
  20: [],
  30: [],
  40: [],
  50: [],
  60: [],
  70: [],
  80: [],
  90: [],
  100: []},
 'falstar': {2: [0.8, 0.8, 0.8, 0.7, 0.7, 0.7, 0.7, 0.7, 0.8, 0.7],
  4: [4.7, 4.6, 5.6, 5.2, 3.2, 1.7, 5.2, 5.3, 5.2, 0.8],
  6: [166.25, 7.7, 3.6, 0.7],
  8: [22.444444444444407, 4.2, 0.8],
  10: [54.28571428571427, 0.8],
  20: [38.74999999999999],
  30: [],
  40: [],
  50: [],
  60: [],
  70: [],
  80: [],
  90: [],
  100: []},
 'enhsp': {2: [0.28107357025146484,
   0.26201963424682617,
   0.24789857864379883,
   0.2833428382873535,
   0.2796633243560791,
   0.28940320014953613,
   0.30948925018310547,
   0.2852311134338379,
   0.2560701370239258,
   0.23923826217651367],
  4: [1.3102412223815918,


## Generate Summary CSV

In [6]:
runtime = open("{}/runtime.csv".format(folder), "w")
coverage = open("{}/coverage.csv".format(folder), "w")

for h in horizons:
    runtime_line = []
    coverage_line = []
    for app in approaches:
        if len(results[app][h]) > 0:
            runtime_line += [sum(results[app][h])/len(results[app][h])]
        else:
            runtime_line += [0]
        coverage_line += [len(results[app][h])]
        
    runtime.write(", ".join(map(str, runtime_line)))
    runtime.write("\n")
    coverage.write(", ".join(map(str, coverage_line)))
    coverage.write("\n")
    
runtime.close()
coverage.close()

## Generate Table

In [7]:
column_names = {"dreach": "\\textsc{dReach}", "falstar": "\\textsc{Falstar}", "enhsp": "ENHSP$_a^{0.01}$", "enhsp_e_001": "ENHSP$_e^{0.01}$", "enhsp_e_01": "ENHSP$_e^{0.1}$", "enhsp_e_04": "ENHSP$_e^{0.4}$"}

table_tex = "Size & {} \\\\\n".format(" & ".join(column_names.values()))

for h in horizons:
    h_line = []
    for app in approaches:
        coverage = len(results[app][h])
        if coverage > 0:
            avg_runtime = sum(results[app][h])/coverage
            h_line += ["{} ({})".format(coverage, round(avg_runtime, 2))]
        else:
            h_line += ["0"]
        
    table_tex += "{} & {} \\\\\n".format(h, " & ".join(h_line))


In [8]:
print(table_tex)

Size & \textsc{dReach} & \textsc{Falstar} & ENHSP$_a^{0.01}$ & ENHSP$_e^{0.01}$ & ENHSP$_e^{0.1}$ & ENHSP$_e^{0.4}$ \\
2 & 10 (0.33) & 10 (0.74) & 10 (0.27) & 10 (0.53) & 10 (0.23) & 10 (0.23) \\
4 & 1 (13.9) & 10 (4.15) & 10 (1.6) & 10 (0.86) & 10 (0.4) & 10 (0.23) \\
6 & 0 & 4 (44.56) & 10 (1.06) & 7 (9.94) & 10 (1.33) & 10 (0.26) \\
8 & 0 & 3 (9.15) & 10 (1.87) & 7 (16.87) & 10 (1.64) & 10 (0.28) \\
10 & 0 & 2 (27.54) & 5 (45.01) & 6 (15.14) & 10 (2.06) & 10 (0.29) \\
20 & 0 & 1 (38.75) & 4 (39.33) & 2 (155.15) & 10 (24.26) & 10 (0.41) \\
30 & 0 & 0 & 0 & 1 (41.69) & 3 (1.39) & 10 (1.26) \\
40 & 0 & 0 & 0 & 0 & 3 (2.09) & 10 (1.8) \\
50 & 0 & 0 & 0 & 0 & 3 (5.99) & 10 (2.52) \\
60 & 0 & 0 & 0 & 0 & 3 (10.33) & 10 (3.04) \\
70 & 0 & 0 & 0 & 0 & 3 (14.71) & 10 (3.91) \\
80 & 0 & 0 & 0 & 0 & 2 (40.5) & 10 (4.53) \\
90 & 0 & 0 & 0 & 0 & 2 (56.57) & 10 (6.0) \\
100 & 0 & 0 & 0 & 0 & 1 (160.04) & 10 (7.47) \\

